In [3]:
import torch
import torch.nn as nn
from torchvision import models

# Load pretrained MobileNetV2
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Freeze all layers initially
for param in model.features.parameters():
    param.requires_grad = False

# Modify classifier (last layer)
num_classes = 3
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\USER/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100%|█████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:00<00:00, 23.5MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [22]:
# ===============================================
# 📥 BLOCK 1: DOWNLOAD DATASET FROM PEXELS
# ===============================================
import os
import requests
import time
import random
from tqdm.notebook import tqdm
from math import floor

# 🔐 Your PEXELS API key
PEXELS_API_KEY = "c0RqbswWMj17E5o2KbQroMPADjcMhoGX3DoKxU5IVkUZkh9DupaarqlJ"

# 📂 Dataset structure
BASE_DIR = r"D:\VED\Coding\ML\MobileNetAI"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")

# 🐶 Define your classification classes
CLASSES = ["dog", "cat", "cow"]
PER_CLASS = 800  # total per class (e.g., 640 train + 160 val)

# 🔗 Pexels API
BASE_URL = "https://api.pexels.com/v1/search"
HEADERS = {"Authorization": PEXELS_API_KEY}

# 🛠️ 1. Create folder structure
def create_base_structure():
    os.makedirs(TRAIN_DIR, exist_ok=True)
    os.makedirs(VAL_DIR, exist_ok=True)
    for cls in CLASSES:
        os.makedirs(os.path.join(TRAIN_DIR, cls), exist_ok=True)
        os.makedirs(os.path.join(VAL_DIR, cls), exist_ok=True)
    print(f"✅ Folder structure created at {BASE_DIR}")

# 📥 2. Fetch images for one class
def fetch_images_for_class(cls, total_images=PER_CLASS):
    temp_dir = os.path.join(BASE_DIR, f"temp_{cls}")
    os.makedirs(temp_dir, exist_ok=True)

    count = 0
    page = 1
    pbar = tqdm(total=total_images, desc=f"Downloading '{cls}'")
    while count < total_images:
        params = {"query": cls, "per_page": 80, "page": page}
        resp = requests.get(BASE_URL, headers=HEADERS, params=params)
        time.sleep(0.5)
        if resp.status_code != 200:
            print(f"❌ Error {resp.status_code} for {cls} page {page}")
            break

        photos = resp.json().get("photos", [])
        if not photos:
            print(f"⚠️ No more photos for {cls}")
            break

        for photo in photos:
            if count >= total_images:
                break
            img_url = photo["src"]["original"]
            img_id = photo["id"]
            ext = img_url.split(".")[-1].split("?")[0]
            filename = f"{img_id}.{ext}"
            save_path = os.path.join(temp_dir, filename)
            try:
                img_data = requests.get(img_url).content
                with open(save_path, "wb") as f:
                    f.write(img_data)
                count += 1
                pbar.update(1)
            except:
                continue
        page += 1

    pbar.close()
    print(f"✅ Downloaded {count} images for '{cls}'")
    return temp_dir

# ✂️ 3. Split into train/val
def split_images(temp_dir, cls):
    files = os.listdir(temp_dir)
    random.shuffle(files)

    train_count = floor(len(files) * 0.8)
    train_files = files[:train_count]
    val_files = files[train_count:]

    train_cls_dir = os.path.join(TRAIN_DIR, cls)
    val_cls_dir = os.path.join(VAL_DIR, cls)

    for f in train_files:
        os.rename(os.path.join(temp_dir, f), os.path.join(train_cls_dir, f))
    for f in val_files:
        os.rename(os.path.join(temp_dir, f), os.path.join(val_cls_dir, f))

    os.rmdir(temp_dir)
    print(f"📂 {cls}: {len(train_files)} train | {len(val_files)} val")

# 🚀 4. Main pipeline
if __name__ == "__main__":
    create_base_structure()
    start = time.time()
    for cls in CLASSES:
        tdir = fetch_images_for_class(cls, PER_CLASS)
        split_images(tdir, cls)
        time.sleep(2)
    print(f"\n✅ Dataset ready in {round(time.time() - start, 2)} seconds")


✅ Folder structure created at D:\VED\Coding\ML\MobileNetAI


✅ Downloaded 800 images for 'dog'
📂 dog: 640 train | 160 val


✅ Downloaded 800 images for 'cat'
📂 cat: 640 train | 160 val


✅ Downloaded 800 images for 'cow'
📂 cow: 640 train | 160 val

✅ Dataset ready in 4836.83 seconds


In [24]:
# ===============================================
# 🧠 BLOCK 2: TRAINING MOBILE NET
# ===============================================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
from datetime import timedelta
from tqdm import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ Using device: {device}")

# Paths
train_dir = os.path.join(BASE_DIR, "train")
val_dir = os.path.join(BASE_DIR, "val")

# Transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Datasets
train_data = datasets.ImageFolder(train_dir, transform=transform_train)
val_data = datasets.ImageFolder(val_dir, transform=transform_val)
class_names = train_data.classes

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print(f"📸 Train: {len(train_data)} | Val: {len(val_data)} | Classes: {class_names}")

# Load MobileNetV2
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
for p in model.features.parameters():
    p.requires_grad = False
model.classifier[1] = nn.Linear(model.last_channel, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0005)

# Training loop
num_epochs = 10
start_time = time.time()

print("🚀 Training started...")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_data)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    val_acc = 100 * correct / total

    elapsed = timedelta(seconds=int(time.time() - start_time))
    print(f"✅ Epoch [{epoch+1}/{num_epochs}] | Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}% | ⏱️ {elapsed}")

torch.save(model.state_dict(), "mobilenet_best.pth")
print("💾 Model saved as mobilenet_best.pth")


⚡ Using device: cuda
📸 Train: 1920 | Val: 480 | Classes: ['cat', 'cow', 'dog']
🚀 Training started...


✅ Epoch [1/10] | Loss: 0.7634 | Val Acc: 83.75% | ⏱️ 0:31:49


✅ Epoch [2/10] | Loss: 0.4600 | Val Acc: 90.62% | ⏱️ 1:04:07


✅ Epoch [3/10] | Loss: 0.3722 | Val Acc: 89.58% | ⏱️ 1:34:49


✅ Epoch [4/10] | Loss: 0.3572 | Val Acc: 88.96% | ⏱️ 2:07:09


...

✅ Epoch [6/10] | Loss: 0.3303 | Val Acc: 90.42% | ⏱️ 3:09:15


✅ Epoch [7/10] | Loss: 0.3031 | Val Acc: 90.42% | ⏱️ 3:40:23


✅ Epoch [8/10] | Loss: 0.2976 | Val Acc: 91.46% | ⏱️ 4:12:00


✅ Epoch [9/10] | Loss: 0.2823 | Val Acc: 88.75% | ⏱️ 4:44:09


✅ Epoch [10/10] | Loss: 0.2792 | Val Acc: 91.04% | ⏱️ 5:14:19
💾 Model saved as mobilenet_best.pth


In [16]:
# --------------------------- Imports ---------------------------
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# ------------------------ Device Setup ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------- Load Trained Model ---------------------
# Make sure to load your model structure before loading weights
from torchvision import models
num_classes = 3
class_names = ['dog', 'cat', 'cow']

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)
model.load_state_dict(torch.load("mobilenet_best.pth", map_location=device))
model.to(device)
model.eval()
print("✅ Model loaded and ready!")

# --------------------- Transform Definition -------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --------------------- Prediction Function --------------------
def predict_image(img_path):
    """Predict class of a single image"""
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)  # (1, C, H, W)

    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
        confidence, pred_class = torch.max(probs, 1)

    predicted_label = class_names[pred_class.item()]
    conf_percent = confidence.item() * 100

    print(f"🖼️ Image: {img_path}")
    print(f"🔮 Predicted Class: {predicted_label} ({conf_percent:.2f}% confidence)")
    return predicted_label, conf_percent

# --------------------- Show Prediction ------------------------
def show_prediction(img_path):
    pred_label, conf = predict_image(img_path)
    

# --------------------- Test Examples --------------------------
# Example 1: Dog image
show_prediction(r"D:\VED\Coding\ML\MobileNetAI\train\cow\46505.jpeg")



✅ Model loaded and ready!
🖼️ Image: D:\VED\Coding\ML\MobileNetAI\train\cow\46505.jpeg
🔮 Predicted Class: cow (97.70% confidence)


In [8]:
# ===============================================
# 🧠 BLOCK 3: CALCULATE MOBILE NET RESOURCES (CLEAN OUTPUT with PrettyTable)
# ===============================================

import torch
# You will need to install the thop library first: pip install thop
# And PrettyTable: pip install prettytable
from thop import profile
from torchvision import models
import time
import torch.nn as nn
import os
from prettytable import PrettyTable # Import PrettyTable

# --- CRITICAL FIX: DEFINE DEVICE ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. Load the exact model configuration ---
# Define placeholders needed to run this block independently
BASE_DIR = os.getenv("BASE_DIR", ".")
CLASSES = ["dog", "cat", "cow"] # Use the actual classes
num_classes = len(CLASSES)
input_size = (3, 224, 224) 

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
# Freeze/Modify layers exactly as done in the training block
for p in model.features.parameters():
    p.requires_grad = False
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

# Load saved weights (Optional, but ensures the reported size is for your trained model)
try:
    # Replace "mobilenet_best.pth" with the full, correct path if running separately
    model.load_state_dict(torch.load("mobilenet_best.pth", map_location=device))
    print("✅ Loaded trained weights from mobilenet_best.pth")
except FileNotFoundError:
    print("⚠️ Weights file not found. Calculating resources for the base MobileNetV2 structure.")
    
model = model.to(device)
model.eval()

# --- 2. Define Dummy Input ---
dummy_input = torch.randn(1, *input_size).to(device)

print("\n" + "="*70)
print(f"{'-- CALCULATING MODEL RESOURCES FOR MOBILE DEPLOYMENT --':^70}")
print("="*70)
start_t_calc = time.time()

# --- 3. Calculate MACs (MAdds) and Parameters ---
macs, params = profile(model, inputs=(dummy_input,), verbose=False)

# Convert to millions for easy reporting
M_MACs = macs / 1_000_000
M_Params = params / 1_000_000
calculation_time = time.time() - start_t_calc

# --- FORMATTED RESOURCE SUMMARY ---
# We use Python's built-in formatting for the summary block
summary = PrettyTable()
summary.field_names = ["Metric", "Value"]
summary.align["Metric"] = "l"
summary.align["Value"] = "r"
summary.add_row(["Input Resolution", f"{input_size[1]}x{input_size[2]}"])
summary.add_row(["Total MACs (MAdds)", f"{M_MACs:.2f} Million"])
summary.add_row(["Total Parameters", f"{M_Params:.2f} Million"])
summary.add_row(["Calculation Time", f"{calculation_time:.4f} seconds"])

print(summary)
print("="*70)


# --- 4. Final Presentation Proof Table (PrettyTable Formatting) ---

# Metrics from the MobileNet papers for comparison
paper_macs = 569   # MobileNetV1 1.0-224 (Million MAdds)
paper_params = 4.2 # MobileNetV1 1.0-224 (Million Parameters)
vgg_macs = 15300   # VGG-16
vgg_params = 138

# Create the final comparison table
comparison_table = PrettyTable()
comparison_table.title = "FINAL PRESENTATION PROOF: MOBILE EFFICIENCY COMPARISON"
comparison_table.field_names = [
    "Metric", 
    "Your MobileNetV2 Model", 
    "Paper's MobileNet (V1)", 
    "VGG-16 (Inefficient)"
]

# Set column alignment
comparison_table.align["Metric"] = "l"
comparison_table.align["Your MobileNetV2 Model"] = "c"
comparison_table.align["Paper's MobileNet (V1)"] = "c"
comparison_table.align["VGG-16 (Inefficient)"] = "c"


# Row 1: Computational Cost (MAdds)
comparison_table.add_row([
    "Computational Cost (MAdds)", 
    f"{M_MACs:.2f} Million",
    f"{paper_macs} Million",
    f"{vgg_macs} Million"
])

# Row 2: Parameters (Size)
comparison_table.add_row([
    "Parameters (Size)", 
    f"{M_Params:.2f} Million",
    f"{paper_params} Million",
    f"{vgg_params} Million"
])

print(comparison_table)

# --- CONCLUSION ---
conclusion = (
    f"Conclusion: Your MobileNetV2 model's computational cost "
    f"({M_MACs:.2f} MAdds) is approximately {vgg_macs / M_MACs:.0f}x lower than the "
    f"VGG-16 architecture, directly proving its viability for resource-constrained "
    f"mobile or embedded devices, fulfilling the project requirements for efficiency. "
    f""
)
print("\n" + conclusion)


✅ Loaded trained weights from mobilenet_best.pth

       -- CALCULATING MODEL RESOURCES FOR MOBILE DEPLOYMENT --        
+--------------------+----------------+
| Metric             |          Value |
+--------------------+----------------+
| Input Resolution   |        224x224 |
| Total MACs (MAdds) | 326.21 Million |
| Total Parameters   |   2.23 Million |
| Calculation Time   | 0.0275 seconds |
+--------------------+----------------+
+-----------------------------------------------------------------------------------------------------+
|                        FINAL PRESENTATION PROOF: MOBILE EFFICIENCY COMPARISON                       |
+----------------------------+------------------------+------------------------+----------------------+
| Metric                     | Your MobileNetV2 Model | Paper's MobileNet (V1) | VGG-16 (Inefficient) |
+----------------------------+------------------------+------------------------+----------------------+
| Computational Cost (MAdds) |     326.